# 06: Procesamiento de Diputados y Biografías

**Propósito:** Este *notebook* toma todos los archivos `diputados_bio.csv` (crudos, por período) de `data/01_raw/`, y los transforma en un único archivo maestro de diputados limpio.

**Proceso:**
1.  **Carga y Consolidación:** Lee todos los archivos `diputados_bio.csv` y los une.
2.  **Limpieza y Estandarización:** Normaliza los campos extraídos por el LLM (ej. `universidad`, `maximo_nivel_educativo`) usando mapeos y *fuzzy matching*.
3.  **Deduplicación:** Crea un registro único por `Diputado.Id`, seleccionando la biografía de mayor calidad (mejor `match_score`).
4.  **Guardado:** Guarda el archivo maestro en `data/02_processed/`.

**Dependencias:**
* `data/01_raw/[periodo]/diputados_bio.csv` (Múltiples archivos)

**Salidas (Artifacts):**
* `data/02_processed/diputados_master_clean.parquet` (Un único archivo)

In [28]:
import pandas as pd
from pathlib import Path
import sys
import logging
from tqdm.notebook import tqdm # Para progress_apply
import numpy as np

# --- Configurar Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Importar lógica personalizada de /src ---
sys.path.append('../') 
try:
    from src.processing_utils import (
        load_all_bio_files, 
        standardize_education,
        deduplicate_deputies,
        standardize_civil_status,
        standardize_location,
        extract_last_colegio,
        standardize_career
    )
    from src.common_utils import normalize_string
except ImportError as e:
    logging.error(f"ERROR: No se pudieron importar las funciones desde /src. {e}")
    raise

# Registrar 'tqdm' con pandas
tqdm.pandas()

In [29]:
# --- 1. Configuración de Rutas y Constantes ---
ROOT = Path.cwd().parent
DATA_DIR_RAW = ROOT / "data" / "01_raw"
DATA_DIR_PROCESSED = ROOT / "data" / "02_processed"

# Asegurarse que el directorio de salida exista
DATA_DIR_PROCESSED.mkdir(parents=True, exist_ok=True)

# Archivo de salida
OUTPUT_FILE = DATA_DIR_PROCESSED / "diputados_master_clean.parquet"

logging.info(f"Directorio Raw: {DATA_DIR_RAW}")
logging.info(f"Directorio Processed: {DATA_DIR_PROCESSED}")
logging.info(f"Archivo de Salida: {OUTPUT_FILE}")

2025-10-28 12:54:58,873 - INFO - Directorio Raw: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw
2025-10-28 12:54:58,873 - INFO - Directorio Processed: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\02_processed
2025-10-28 12:54:58,874 - INFO - Archivo de Salida: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\02_processed\diputados_master_clean.parquet


## 1. Carga y Consolidación

Cargamos todos los archivos `diputados_bio.csv` de todos los períodos en un solo DataFrame.

In [30]:
# Llamar a la función del módulo /src
df_full = load_all_bio_files(DATA_DIR_RAW)

if not df_full.empty:
    display(df_full.head())
    print(f"Dimensiones del DataFrame consolidado: {df_full.shape}")
else:
    logging.error("No se cargaron datos. Deteniendo el notebook.")
    # raise Exception("No se cargaron datos.")

2025-10-28 12:54:59,669 - INFO - Buscando archivos 'diputados_bio.csv'...
2025-10-28 12:54:59,671 - INFO - Encontrados 7 archivos. Cargando...
2025-10-28 12:54:59,747 - INFO - DataFrame consolidado creado con 929 filas.


,FechaInicio,FechaTermino,Diputado.Id,Diputado.Nombre,Diputado.Nombre2,Diputado.ApellidoPaterno,Diputado.ApellidoMaterno,Diputado.FechaNacimiento,Diputado.FechaDefucion,Diputado.RUT,...,madre,estado_civil,numero_total_hijos,colegios,universidad,carrera,maximo_nivel_educativo,trabajo,fuente_periodo,Distrito
0,2002-03-10,NaN,1,Mario,NaN,Acuña,Cisternas,NaN,NaN,NaN,...,María Cisterna Fuentealba,Casado/a,3.0,[],Universidad Austral de Chile,Ingeniero Agrónomo,Educación Universitaria,[],1998-2002,NaN
1,2002-03-10,NaN,3,Gustavo,NaN,Alessandri,Valdés,NaN,NaN,NaN,...,Verónica Balmaceda,Casado/a,3.0,['Colegio de los Sagrados Corazones de Manqueh...,Universidad de California,Maquinaria agrícola,Educación Universitaria,"['Obrero', 'Empresario independiente', 'Propie...",1998-2002,NaN
2,2002-03-10,NaN,8,Rafael,NaN,Arratia,Valdebenito,NaN,NaN,NaN,...,Olivia Valdebenito Cuevas,Casado/a,4.0,['Colegio Manuel León Prado'],Universidad de Chile,Médico cirujano,Educación Universitaria,"['Médico oftalmólogo', 'Director Médico de la ...",1998-2002,NaN
3,2002-03-10,NaN,10,Nelson,NaN,Avila,Contreras,NaN,NaN,NaN,...,María Olivia Contreras Chinchón,Casado,2.0,"['Colegio Las Carmelitas', 'Liceo de Hombres d...",Universidad de Chile,Administrador Público,Educación Universitaria,['Jefe de personal en Entel Chile'],1998-2002,NaN
4,2002-03-10,NaN,11,Francisco,NaN,Bartolucci,Johnston,NaN,NaN,NaN,...,Josefina Jhonston Miranda,Casado/a,5.0,['Colegio San Pedro Nolasco'],Universidad Católica de Valparaíso,Ciencias Jurídicas y Sociales,Educación Universitaria,['Profesor auxiliar de Derecho Romano'],1998-2002,NaN


Dimensiones del DataFrame consolidado: (929, 43)


## 2. Limpieza y Estandarización

Aplicamos las funciones de limpieza de `src/processing_utils.py` para normalizar los campos extraídos por el LLM.

In [31]:
if not df_full.empty:
    # --- Limpieza de Tipos ---
    df_processed = df_full.copy()
    logging.info("Limpiando tipos de datos...")
    df_processed['fecha_nacimiento'] = pd.to_datetime(df_full['fecha_nacimiento'], errors='coerce')
    df_processed['numero_total_hijos'] = pd.to_numeric(df_full['numero_total_hijos'], errors='coerce').astype('Int64')
    
    df_processed = standardize_education(df_processed)
    df_processed = standardize_career(df_processed)
    df_processed['fecha_nacimiento_llm'] = pd.to_datetime(df_processed['fecha_nacimiento'], errors='coerce')
    df_processed['fecha_nacimiento_api'] = pd.to_datetime(df_processed['Diputado.FechaNacimiento'], errors='coerce')
    df_processed['fecha_nac_clean'] = df_processed['fecha_nacimiento_api'].fillna(
            df_processed['fecha_nacimiento_llm']
        )
    df_processed['colegio_egreso_raw'] = df_processed['colegios'].apply(extract_last_colegio)
    df_processed['estado_civil_clean'] = df_processed['estado_civil'].progress_apply(lambda x: standardize_civil_status(pd.Series(x)))
    df_location_features = standardize_location(df_processed['lugar_nacimiento'])
    df_processed = df_processed.join(df_location_features)
    df_processed['colegio_egreso_merge_key'] = df_processed['colegio_egreso_raw'].apply(
        lambda x: np.nan if pd.isna(x) else normalize_string(x)
    )
    
    print(df_processed.columns)
    logging.info("Procesamiento de campos finalizado.")
    display(df_processed[['universidad', 'universidad_clean', 'maximo_nivel_educativo', 'educacion_nivel_clean']].sample(10))
    COLUMNAS_FINALES = ['FechaInicio', 'Diputado.Id', 'Diputado.Nombre',
       'Diputado.ApellidoPaterno',
       'Diputado.ApellidoMaterno',
       'Diputado.Sexo._value_1', 'Diputado.Sexo.Valor',
       'Diputado.Militancias.Militancia', 'Distrito.Numero',
       'Distrito.Comunas.Comuna', 'FechaInicio.1', 'FechaTermino.1',
       'Partido.Nombre', 'Partido.Alias', 'match_nombre_bcn', 'padre', 'madre', 'numero_total_hijos',
       'fuente_periodo' 'educacion_nivel_clean',
       'universidad_clean', 'universidad_tipo', 'carrera_clean_list',
       'carrera_clean_1', 'carrera_clean_2', 'fecha_nac_clean', 'estado_civil_clean', 'ciudad_nac', 'pais_nac',
       'colegio_egreso_merge_key']
    df_master = df_processed.drop(columns=to_drop).reset_index(drop=True)
else:
    logging.warning("DataFrame vacío, saltando limpieza.")

2025-10-28 12:55:00,589 - INFO - Limpiando tipos de datos...
2025-10-28 12:55:00,595 - INFO - Estandarizando 'maximo_nivel_educativo'...
2025-10-28 12:55:00,599 - INFO - Estandarizando 'universidad' con estrategia 'map-once'...
2025-10-28 12:55:00,600 - INFO - Se encontraron 118 valores únicos de universidad.
2025-10-28 12:55:00,601 - INFO - Construyendo mapa de traducción (Manual + Fuzzy)...
Creando Mapa Fuzzy: 100%|██████████████████████████████████████████████████████████| 118/118 [00:00<00:00, 2447.57it/s]
2025-10-28 12:55:00,653 - INFO - Aplicando mapa a todas las filas...
2025-10-28 12:55:00,654 - INFO - Clasificando tipo de universidad...
2025-10-28 12:55:00,659 - INFO - Estandarizando 'carrera' con estrategia 'map-once'...
2025-10-28 12:55:00,660 - INFO - Se encontraron 212 valores únicos de carrera.
2025-10-28 12:55:00,660 - INFO - Construyendo mapa de traducción de carreras (Regex)...
Mapeando Carreras: 100%|██████████████████████████████████████████████████████████| 212/212 

  0%|          | 0/929 [00:00<?, ?it/s]

2025-10-28 12:55:01,764 - INFO - Procesamiento de campos finalizado.


Index(['FechaInicio', 'FechaTermino', 'Diputado.Id', 'Diputado.Nombre',
       'Diputado.Nombre2', 'Diputado.ApellidoPaterno',
       'Diputado.ApellidoMaterno', 'Diputado.FechaNacimiento',
       'Diputado.FechaDefucion', 'Diputado.RUT', 'Diputado.RUTDV',
       'Diputado.Sexo._value_1', 'Diputado.Sexo.Valor',
       'Diputado.Militancias.Militancia', 'Distrito.Numero',
       'Distrito.Comunas.Comuna', 'FechaInicio.1', 'FechaTermino.1',
       'Partido.Id', 'Partido.Nombre', 'Partido.Alias', 'nombre_completo',
       'match_nombre_bcn', 'match_score', 'url_wiki', 'status', 'distrito',
       'familia_juventud_parrafos', 'estudios_vida_laboral_parrafos',
       'bio_texto_completo', 'lugar_nacimiento', 'fecha_nacimiento', 'padre',
       'madre', 'estado_civil', 'numero_total_hijos', 'colegios',
       'universidad', 'carrera', 'maximo_nivel_educativo', 'trabajo',
       'fuente_periodo', 'Distrito', 'educacion_nivel_clean',
       'universidad_clean', 'universidad_tipo', 'carrera_cle

,universidad,universidad_clean,maximo_nivel_educativo,educacion_nivel_clean
357,Universidad Católica de Chile,Pontificia Universidad Católica de Chile,Educación Universitaria,Universitaria
425,Universidad de Chile,Universidad de Chile,Educación Universitaria,Universitaria
164,Universidad de Chile,Universidad de Chile,Educación Universitaria,Universitaria
780,Universidad de Concepción,Universidad de Concepción,Educación Universitaria,Universitaria
583,Universidad Austral de Chile,Universidad Austral de Chile,Educación Universitaria,Universitaria
101,Universidad de Chile,Universidad de Chile,NaN,Universitaria
122,Pontificia Universidad Católica de Chile,Pontificia Universidad Católica de Chile,Educación Universitaria,Universitaria
56,Universidad de Chile,Universidad de Chile,Educación Universitaria,Universitaria
354,Universidad de Chile,Universidad de Chile,Educación Universitaria,Universitaria
726,Universidad Austral de Chile,Universidad Austral de Chile,Educación Universitaria,Universitaria
